In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import copy
import cv2
import math
import shutil
from PIL import Image
from sklearn.metrics import accuracy_score
from IPython.display import Video, display
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial import KDTree
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
import random
import sys
# sys.path.append('../input/easydict-master/easydict-master/')
# # https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch
# sys.path.append('../input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch/')
# from deep_sort.deep_sort import DeepSort
# from utils.parser import get_config
import warnings
warnings.simplefilter('ignore')

In [ ]:
## Install helmet-assignment helper code
!pip install ../input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features
from helmet_assignment.video import video_with_predictions

In [ ]:
! pip install ../input/pip-install-sort/filterpy-1.4.5.zip
sys.path.append('../input/sort-github/sort-master/')
from sort import Sort

# Functions
## line param pred

In [ ]:
def calc_dif_angle(theta1, theta2):
    theta_large, theta_small = np.zeros_like(theta1), np.zeros_like(theta2)
    theta_large = np.max(np.array([theta1, theta2]), axis=0)
    theta_small = np.min(np.array([theta1, theta2]), axis=0)

    dif1 = theta_large - theta_small
    dif2 = np.pi + theta_small - theta_large
    return np.min(np.array([dif1, dif2]), axis=0)

def calc_loss_angles(angles, target_angle, thres = 0.4):
    dif = calc_dif_angle(angles, np.ones_like(angles) * target_angle)
    loss = dif * dif
    loss[dif > thres] = np.abs(dif[dif > thres]) # Huber loss
    return np.sum(loss)

def calc_optimal_angle(angles):
    angle_opt = 0
    loss_opt = 1e9
    for ang in np.linspace(0, np.pi, 100):
        loss = calc_loss_angles(angles, ang)
        if loss < loss_opt:
            loss_opt = loss
            angle_opt = ang
    return angle_opt

def loss_func(cand_scale, dif_list):
    loss_residual = np.sum([
        np.min([np.abs(dif % cand_scale), np.abs(cand_scale - dif % cand_scale)]
              ) ** 2 for dif in dif_list]) # 割ったあまり
    loss_multipled = np.sum([np.max(dif // cand_scale - 1, 0) for dif in dif_list]) # n倍なら、n-1を罰則として与える（n >= 1ならなし）
    loss_under = np.sum([np.max(cand_scale // dif - 1, 0) for dif in dif_list]) # 1/n倍なら、n-1を罰則として与える（n <= 1ならなし）
    return loss_residual + loss_multipled + loss_under

def get_optimal_dif(dif_list, view):
    opt_dif = 0
    opt_loss = 1e9
    if view == 'Sideline':
        candidates = np.arange(100, 250, 5)
    else:
        candidates = np.arange(70, 170, 5)
        
    for x in candidates:
        loss = loss_func(x, dif_list)
        if loss < opt_loss:
            opt_dif = x
            opt_loss = loss
    return opt_dif

def get_average_dist_from_lines(line_pos_list, view):
    sets = []
    min_thres = 30
    for d in sorted(line_pos_list):
        if len(sets) == 0:
            sets.append([d])
        else:
            if d - sets[-1][-1] < min_thres:
                sets[-1].append(d)
            else:
                sets.append([d])
    pos_list = np.array([np.mean(set_pos) for set_pos in sets])
    
    if True:
        dist_pred = get_optimal_dif(pos_list[1:] - pos_list[:-1], view)
    else:
        dist_pred = np.mean(pos_list[1:] - pos_list[:-1])
    # print(*pos_list, sep=',')
    # print(*pos_list[1:] - pos_list[:-1], sep=',')
    return dist_pred

def predict_line_parameters(img, view = 'Endzone'):
    kernel_size = 40
    img_dif_threshold = 70

    kernel = np.ones((kernel_size, kernel_size)) / kernel_size **2
    img_morph = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    counter = 0

    while True:
        img_dif = np.linalg.norm(img_morph - np.median(img, axis=(0, 1)), axis=2)
        img_field = np.copy(img)
        img_field[img_dif > img_dif_threshold - counter * 10] = 0
        edges = cv2.Canny(img_field, 50, 150, apertureSize = 3)
        lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
        if lines is None:
            return np.nan, np.nan
        if len(lines) < 100:
            break
        counter += 1
        if img_dif_threshold - counter * 10 < 0:
            return np.nan, np.nan
    
    if lines is None:
        return np.nan, np.nan
    if len(lines) > 100:
        return np.nan, np.nan

    # Drop duplicates
    lines_new = np.copy(lines)
    thres_angle = 0.1
    thres_dist = 20
    idx = 0
    while len(lines_new) > idx:
        dif_dist = lines_new[:, 0, 0] - lines_new[idx, 0, 0]
        dif_angle = calc_dif_angle(lines_new[:, 0, 1], np.ones_like(lines_new[:, 0, 1]) * lines_new[idx, 0, 1])
        duplicate = (dif_dist < thres_dist) & (dif_angle < thres_angle) & (dif_dist > 0) & (dif_angle > 0)
        lines_new = lines_new[~duplicate]
        idx += 1

    if view == 'Sideline':
        lines_new = lines_new[np.abs(lines_new[:, 0, 1] - np.pi / 2) > np.pi / 4]
    else:
        lines_new = lines_new[np.abs(lines_new[:, 0, 1] - np.pi / 2) < np.pi / 4]
    lines_new = lines_new[lines_new[:, 0, 1] != 0] # Remove edges

    # Calculate angles (Took a lot of time defining optimal angle...)
    theta_pred = calc_optimal_angle(lines_new[:, 0, 1])
    lines_same_direction = lines_new[calc_dif_angle(lines_new[:, 0, 1], np.ones_like(lines_new[:, 0, 1]) * theta_pred) < 0.4]
    if len(lines_same_direction) == 0:
        return np.nan, np.nan

    if view == 'Sideline':
        ym = 360
        rho = lines_same_direction[:, 0, 0]
        theta = lines_same_direction[:, 0, 1]
        xms = (rho - ym * np.sin(theta)) / np.cos(theta)
        dist_pred = get_average_dist_from_lines(xms, view) * np.abs(np.cos(theta_pred))
    else:
        xm = 640
        rho = lines_same_direction[:, 0, 0]
        theta = lines_same_direction[:, 0, 1]
        yms = (rho - xm * np.cos(theta)) / np.sin(theta)
        dist_pred = get_average_dist_from_lines(yms, view) * np.abs(np.sin(theta_pred))

    return theta_pred, dist_pred

## icp

In [ ]:
from scipy.spatial import KDTree
import numpy as np

class ICP(object):
    def __init__(self, pointsA, pointsB):
        self.pointsA = pointsA
        self.pointsB = pointsB
        self.kdtree = KDTree(self.pointsA)
        self.loss = 0

    def calculate(self, iter):
        old_points = np.copy(self.pointsB)
        new_points = np.copy(self.pointsB)

        try:
            for i in range(iter):
                neighbor_idx = self.kdtree.query(old_points)[1]
                targets = self.pointsA[neighbor_idx]
                R, T = calcRigidTranformation(old_points, targets)
                new_points = np.dot(R, old_points.T).T + T
                if  np.sum(np.abs(old_points - new_points)) < 1e-10:
                    break

                old_points = np.copy(new_points)

                self.loss = np.mean(np.linalg.norm(targets - new_points, axis=1))
        except:
            return new_points

        return new_points

def calcRigidTranformation(MatA, MatB):
    A, B = np.copy(MatA), np.copy(MatB)

    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)

    A -= centroid_A
    B -= centroid_B

    H = np.dot(A.T, B)
    U, S, V = np.linalg.svd(H)
    R = np.dot(V.T, U.T)
    T = np.dot(-R, centroid_A) + centroid_B

    return R, T

In [ ]:
def calculate_hungarian_matching(pos_helmets_input, pos_tracking_input, rate_x2y,
                                 # conf_helmets,
                                ):
    pos_helmets = np.copy(pos_helmets_input)
    pos_tracking = np.copy(pos_tracking_input)
    pos_helmets[:, 0] *= rate_x2y
    pos_tracking[:, 0] *= rate_x2y
    dist_matrix = distance.cdist(pos_helmets, pos_tracking, metric='euclidean')
    # dist_matrix /= conf_helmets.reshape(-1, 1)
    helmets_idx, tracking_idx = linear_sum_assignment(dist_matrix)
    dist = np.array([dist_matrix[i, j] for i, j in zip(helmets_idx, tracking_idx)])
    return dist, helmets_idx, tracking_idx

In [ ]:
def calculate_hungarian_matching_with_team(
        pos_helmets_input,
        pos_tracking_input,
        rate_x2y,
        # conf_helmets,
        team_helmets,
        labels_tracking,
        team_dif_penalty,
    ):
    """
    pos_helmets: np.ndarray
    pos_tracking: np.ndarray
    rate_x2y: float
    team_helmets: [0 or 1]
    team_tracking: ['Hn' for 'Vn']
    """
    dist_sum_opt = 1e9
    dist_opt = None
    helmets_idx_opt = None
    tracking_idx_opt = None
    team_cand = ['H', 'V']

    for cand_1 in team_cand:
        team_tracking = np.array([cand_1 in label for label in labels_tracking]) * 1
        pos_helmets = np.copy(pos_helmets_input)
        pos_tracking = np.copy(pos_tracking_input)
        pos_helmets[:, 0] *= rate_x2y
        pos_tracking[:, 0] *= rate_x2y
        dist_matrix = distance.cdist(pos_helmets, pos_tracking, metric='euclidean')
        for i in range(len(pos_helmets)):
            for j in range(len(pos_tracking)):
                if team_helmets[i] != team_tracking[j]:
                    # dist_matrix[i, j] *= team_dif_penalty
                    # dist_matrix[i, j] += team_dif_penalty
                    dist_matrix[i, j] = np.sqrt(dist_matrix[i, j] ** 2 + team_dif_penalty ** 2)
        # dist_matrix /= conf_helmets.reshape(-1, 1)
        helmets_idx, tracking_idx = linear_sum_assignment(dist_matrix)
        dist = np.array([dist_matrix[i, j] for i, j in zip(helmets_idx, tracking_idx)])

        if np.sum(dist) < dist_sum_opt:
            dist_sum_opt = np.sum(dist)
            dist_opt = dist
            helmets_idx_opt = helmets_idx
            tracking_idx_opt = tracking_idx

    return dist_opt, helmets_idx_opt, tracking_idx_opt

In [ ]:
def get_icp_results(pos_helmets, pos_tracking, theta_init, initial_scale_x, initial_scale_y):
    pos_helmets_scaled = np.copy(pos_helmets)
    pos_helmets_scaled[:, 0] = pos_helmets_scaled[:, 0] * initial_scale_x
    pos_helmets_scaled[:, 1] = pos_helmets_scaled[:, 1] * initial_scale_y

    initial_rot = np.array([[np.cos(theta_init), np.sin(theta_init)],
                            [-np.sin(theta_init), np.cos(theta_init)]])
    pos_helmets_initialized = pos_helmets_scaled @ initial_rot
    pos_helmets_initialized -= np.mean(pos_helmets_initialized, axis=0) - np.mean(pos_tracking, axis=0)

    icp = ICP(pos_tracking, pos_helmets_initialized)
    pos_helmets_icp = icp.calculate(3000)

    return pos_helmets_icp, icp.loss

In [ ]:
class MonteCarloICP:
    def __init__(self, initial_scale_factor, view, num_iter, rate_x2y):
        self.scale_factors = initial_scale_factor
        self.num_iter = num_iter
        self.view = view
        self.rate_x2y = rate_x2y

    def get_icp_results_montecarlo(self,
                                   pos_helmets_input,
                                   pos_tracking_input,
                                   theta_init,
                                   # conf_helmets,
                                   tracking_orientation,
                                   helmets_scale,
                                   helmets_cluster=None,
                                   labels_tracking=None):
        # ###### PARAMETERS ######
        # pixel_per_line_cand_sideline=np.hstack([np.arange(150, 250, 10), np.arange(250, 701, 50)])
        # # pixel_per_line_cand_sideline=np.arange(150, 250, 10)
        # pixel_per_line_cand_endzone=np.hstack([np.arange(60, 160, 15), np.arange(200, 401, 40)])
        # # pixel_per_line_cand_endzone=np.arange(60, 160, 15)
        # ###### PARAMETERS ######


        opt_loss = 1e9
        pos_helmets_opt = None
        self.opt_params = None # Dont forget to initialize!!!!
        for i in range(self.num_iter):
            if self.view == 'Sideline':
                # pixel_per_line = random.choice(PIXEL_PER_LINE_CAND_SIDELINE)
                pixel_per_line = random.choice(LINE_PER_HELMET_CAND_SIDELINE * helmets_scale)
            else:
                # pixel_per_line = random.choice(PIXEL_PER_LINE_CAND_ENDZONE)
                pixel_per_line = random.choice(LINE_PER_HELMET_CAND_ENDZONE * helmets_scale)
            # pixel_per_line = pixel_per_line_init
            
            if np.isnan(theta_init):
                if self.view == "Sideline":
                    theta_init = random.gauss(0, np.pi / 8)
                else:
                    theta_init = random.gauss(np.pi / 2, np.pi / 8)

            initial_scale_x = self.scale_factors[0] / pixel_per_line * np.exp(np.log(SCALE_RATE_EXP) * (0.5 - np.random.random()) * 2)
            initial_scale_y = self.scale_factors[1] / pixel_per_line * np.exp(np.log(SCALE_RATE_EXP) * (0.5 - np.random.random()) * 2)
            forward_shift_scale = random.random() * 0.4

            # Copy variables
            pos_tracking = np.copy(pos_tracking_input)
            pos_helmets = np.copy(pos_helmets_input)

            pos_tracking += forward_shift_scale * np.array([
                np.cos((- tracking_orientation + 90) / 180 * np.pi),
                np.sin((- tracking_orientation + 90) / 180 * np.pi)]
            ).T

            pos_helmets_icp, icp_loss = get_icp_results(
                pos_helmets, pos_tracking,
                theta_init, initial_scale_x, initial_scale_y)

            if helmets_cluster is None:
                dist, helmets_idx, tracking_idx = calculate_hungarian_matching(
                    pos_helmets_icp,
                    pos_tracking,
                    self.rate_x2y,
                    # conf_helmets,
                )
            else:
                dist, helmets_idx, tracking_idx = calculate_hungarian_matching_with_team(
                    pos_helmets_icp,
                    pos_tracking,
                    self.rate_x2y,
                    # conf_helmets,
                    helmets_cluster,
                    labels_tracking,
                    TEAM_DIF_PENALTY
                )
            score = np.sum(dist)
            if score < opt_loss:
                opt_loss = score
                pos_helmets_opt = pos_helmets_icp
                self.opt_params = (initial_scale_x, initial_scale_y, forward_shift_scale)
            
        return pos_helmets_opt, opt_loss

In [ ]:
def apply_icp_per_frame_by_team(args, predict_optimal_theta=False):
    # ###### PARAMETERS ######
    # num_particles=100
    # rate_x2y={'Endzone': 0.5, 'Sideline': 1.0}
    # scale_factor={'Endzone': [2, 4.5], 'Sideline': [4, 10]}
    # team_dif_penalty=2.0
    # ###### PARAMETERS ######

    game_play, view, frame, tracking_frame, helmets_frame, theta_line, video_dir = args
    tracking_frame = tracking_frame[~tracking_frame.duplicated('player')] # drop duplicates!

    pos_tracking = tracking_frame[['x', 'y']].values
    labels_tracking = tracking_frame['player'].values
    helmets_frame = helmets_frame[helmets_frame['conf'] > CONF_THRE]
    pos_helmets = helmets_frame[['x', 'y']].values
    pos_helmets[:, 0] *= -1

    if 'cluster_id' not in helmets_frame.columns:
        helmets_frame['cluster_id'] = -1

    # Get theta pred from image
    if predict_optimal_theta:
        ### FFMPEG MODE ###
        image = Image.open(f'/kaggle/working/temp/{frame}.png')
        image = np.array(image)

        # ### NORMAL EXTRACTION MODE ###
        # cap = cv2.VideoCapture(f'{video_dir}/{game_play}_{view}.mp4')
        # cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        # success, image = cap.read()

        theta_line, pixel_per_line = predict_line_parameters(image, view=view)        
        if np.isnan(theta_line):
            if view == 'Sideline':
                theta_cand_list = list(np.linspace(- np.pi / 4, np.pi / 4, 5))
                theta_cand_list += list(np.linspace(- np.pi / 4 + np.pi, np.pi / 4 + np.pi, 5))
            else:
                theta_cand_list = list(np.linspace(np.pi / 4, 3 * np.pi / 4, 5))
                theta_cand_list += list(np.linspace(np.pi / 4 + np.pi, 3 * np.pi / 4 + np.pi, 5))
        else:
            theta_cand_list = [
                theta_line,
                np.pi + theta_line
            ]
    else:
        theta_cand_list = [theta_line]

    # Start ICP
    micp = MonteCarloICP(SCALE_FACTOR[view], view, num_iter=NUM_PARTICLES, rate_x2y=RATE_X2Y[view])
    opt_loss = 1e9
    pos_helmets_opt = None
    idx_of_interest = [True for _ in pos_helmets]
    for i, theta_cand in enumerate(theta_cand_list):
        pos_helmets_icp, icp_loss = micp.get_icp_results_montecarlo(
            pos_helmets,
            pos_tracking,
            theta_cand,
            # helmets_frame['conf'].values,
            tracking_frame['o'].values,
            helmets_scale = (helmets_frame['width'].mean() + helmets_frame['height'].mean()) / 2,
            helmets_cluster = helmets_frame['cluster_id'].iloc[idx_of_interest].values if helmets_frame['cluster_id'].nunique() > 1 else None,
            labels_tracking = labels_tracking
        )
        if opt_loss > icp_loss:
            opt_loss = icp_loss
            pos_helmets_opt = pos_helmets_icp
            opt_micp = micp
            opt_theta = theta_cand

    # if len(pos_helmets) > NUM_HELMETS_THRES_FOR_OUTSIDERS:
    if helmets_frame[helmets_frame['conf'] > CONF_THRE].shape[0] > NUM_HELMETS_THRES_FOR_OUTSIDERS:
        # print('Try here too')
        counter = 0
        while counter < NUM_ITER_OUTSIDERS:
            for i, theta_cand in enumerate(theta_cand_list):
                thres_y_top = random.choice(np.arange(40, 201, 40))
                thres_y_bottom = 720 - random.choice(np.arange(40, 201, 40))
                idx_of_interest_tmp = (pos_helmets[:, 1] > thres_y_top) & (pos_helmets[:, 1] < thres_y_bottom)
                if np.sum(idx_of_interest_tmp) < 15:
                    continue
                pos_helmets_without_edges = pos_helmets[idx_of_interest_tmp]

                pos_helmets_icp, icp_loss = micp.get_icp_results_montecarlo(
                    pos_helmets_without_edges,
                    pos_tracking,
                    theta_cand,
                    # helmets_frame['conf'].values[idx_of_interest_tmp],
                    tracking_frame['o'].values,
                    helmets_scale = (helmets_frame['width'].mean() + helmets_frame['height'].mean()) / 2,
                    helmets_cluster = helmets_frame['cluster_id'].iloc[idx_of_interest_tmp].values if helmets_frame['cluster_id'].nunique() > 1 else None,
                    labels_tracking = labels_tracking
                )
                if opt_loss > icp_loss:
                    opt_loss = icp_loss
                    pos_helmets_opt = pos_helmets_icp
                    opt_micp = micp
                    idx_of_interest = idx_of_interest_tmp
                    opt_theta = theta_cand
            counter += 1

    # Post Processing
    pos_tracking_tmp = pos_tracking + opt_micp.opt_params[2] * np.array([
        np.cos((- tracking_frame['o'] + 90) / 180 * np.pi),
        np.sin((- tracking_frame['o'] + 90) / 180 * np.pi)]
    ).T

    if helmets_frame['cluster_id'].nunique() == 1:
        dist, helmets_idx, tracking_idx = calculate_hungarian_matching(
           pos_helmets_opt,
           pos_tracking_tmp,
           RATE_X2Y[view],
           # helmets_frame['conf'].values[idx_of_interest],
        )
    else:
        dist, helmets_idx, tracking_idx = calculate_hungarian_matching_with_team(
            pos_helmets_opt,
            pos_tracking_tmp,
            RATE_X2Y[view],
            # helmets_frame['conf'].values[idx_of_interest],
            helmets_frame['cluster_id'].values[idx_of_interest],
            labels_tracking,
            TEAM_DIF_PENALTY
        )
    labels_predicted = labels_tracking[tracking_idx] # [idx_of_interest]
    helmets_pred_frame = helmets_frame.iloc[idx_of_interest].iloc[helmets_idx].copy()
    helmets_pred_frame['label'] = labels_predicted
    helmets_pred_frame['opt_loss'] = opt_loss
    helmets_pred_frame['matching_dist'] = dist

    if predict_optimal_theta:
        return opt_theta
    else:
        return helmets_pred_frame

In [ ]:
def apply_icp_multiprocess_by_team(helmets, tracking_processed, video_dir):
    ###### PARAMETERS ######
    clusterization_rate_thres=1e9
    clusterization_dist_thres=0
    clusterization_helmet_shrink_rate=0.7
    ###### PARAMETERS ######

    df_results_icp_list = []
    df_results_icp = pd.DataFrame()
    game_play2tracking = dict(tuple(tracking_processed.groupby('game_play')))
    for video, helmets_video in helmets.groupby('video'):
        game_play = video.split('_')[0] + '_' + video.split('_')[1]
        view = video.split('_')[2].split('.')[0]
        tracking_video = game_play2tracking[game_play] # tracking_processed[tracking_processed['game_play'] == game_play]
        helmets_video = helmets_video[helmets_video['conf'] > CONF_THRE]
        video_name = f'{game_play}_{view}'

        ### FFMPEG ###
        video_file = f'{video_dir}/{video_name}.mp4'
        if os.path.exists('/kaggle/working/temp'):
            shutil.rmtree('/kaggle/working/temp')
        os.mkdir('/kaggle/working/temp')
        !ffmpeg \
            -hide_banner \
            -loglevel fatal \
            -nostats \
            -i $video_file temp/%d.png
        ### FFMPEG ###

        helmets_clusterized_video = helmet_clusterization_video(helmets_video,
                                                              clusterization_rate_thres,
                                                              clusterization_dist_thres,
                                                              clusterization_helmet_shrink_rate)
        p = Pool(processes=4)
        submission_df_list = []

        tracking_video_dict = dict(list(tracking_video.groupby('frame')))
        helmets_video_dict = dict(list(helmets_clusterized_video.groupby('frame')))

        args_frame_1 = (game_play, view, 1, tracking_video_dict[1], helmets_video_dict[1], np.nan, video_dir)
        optimal_theta = apply_icp_per_frame_by_team(args_frame_1, predict_optimal_theta=True)

        data_video_list = [(game_play, view, int(k), tracking_video_dict[k], helmets_video_dict[k], optimal_theta, video_dir) \
                           for k in helmets_video_dict.keys()]
        with tqdm(total=len(data_video_list)) as pbar:
            for this_df in p.imap(apply_icp_per_frame_by_team, data_video_list):
                df_results_icp_list.append(this_df)
                pbar.update(1)
        p.close()

        ### Delete the results of FFMPEG ###
        shutil.rmtree('/kaggle/working/temp')
        ### Delete the results of FFMPEG ###

    df_results_icp = pd.concat(df_results_icp_list, axis=0)
    return df_results_icp

In [ ]:
def helmet_clusterization_video(helmets_video, rate_thres, dist_thres, helmet_shrink_rate, using_gt=False):
    def return_mean_helmets(d):
        radius_x = d.height / 2 * helmet_shrink_rate 
        radius_y = d.width / 2 * helmet_shrink_rate
        center_x = d.top + d.height / 2
        center_y = d.left + d.width / 2
        return np.mean(image[max([int(center_x - radius_x), 0]): int(center_x + radius_x),
                                                max([int(center_y - radius_y), 0]): int(center_y + radius_y)], axis=(0, 1))
        # return np.mean(image[d.top: d.top + d.height, d.left: d.left + d.width,], axis=(0, 1)), axis=1)

    helmets_info_video_list = []
    helmets_ans_video_list = []
    for frame in sorted(helmets_video['frame'].unique()):
        helmets_frame = helmets_video[helmets_video['frame'] == frame]

        image = Image.open(f'/kaggle/working/temp/{frame}.png')
        image = np.array(image)
        image[:, :, 1] = 0
        # helmets_info_frame = helmets_frame.apply(lambda d: np.mean(
        #     image[np.max([int(d.y - radius), 0]): int(d.y + radius),
        #           np.max([int(d.x - radius), 0]): int(d.x + radius)],
        #     axis=(0, 1)), axis=1)
        # helmets_info_frame = helmets_frame.apply(lambda d: np.mean(
        #    image[d.top: d.top + d.height, d.left: d.left + d.width,],
        #     axis=(0, 1)), axis=1)
        helmets_info_frame = helmets_frame.apply(return_mean_helmets, axis=1)
        helmets_info_frame = np.array([data for data in helmets_info_frame.values])
        helmets_info_video_list.append(helmets_info_frame)
        if using_gt:
            helmets_ans_video_list.append(helmets_frame['label'].apply(lambda x: 'H' in x).values * 1)

    helmets_info_video = np.vstack(helmets_info_video_list)
    if using_gt:
        helmets_ans_video = np.hstack(helmets_ans_video_list)

    try:
        pred = KMeans(n_clusters=2).fit_predict(helmets_info_video)
    except:
        print('DETECTED ERROR IN Kmeans!!!!! OH NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        import traceback
        traceback.print_exc()
        df_tmp = helmets_video.copy()
        df_tmp['cluster_id'] = -1
        return df_tmp

    same_rate = np.sum(pred) / np.sum(1 - pred)
    dist = np.linalg.norm(
        helmets_info_video[pred == 1].mean(axis=0) - helmets_info_video[pred == 0].mean(axis=0)
    )

    df_tmp = helmets_video.copy()
    df_tmp['cluster_id'] = pred

    detected_anomaly = False
    if dist < dist_thres:
        detected_anomaly = True
    if same_rate > rate_thres or same_rate < 1 / rate_thres:
        detected_anomaly = True

    if detected_anomaly:
        df_tmp['cluster_id'] = -1

    print(helmets_video['video'].unique()[0],
          df_tmp['cluster_id'].sum(),
          df_tmp['cluster_id'].shape[0] - df_tmp['cluster_id'].sum(),
          dist)
    if detected_anomaly:
        print('ANOMALY DETECTED!')
    else:
        print('Use team info for this video')

    if using_gt:
        acc_rate = (helmets_ans_video == pred).sum() / len(pred)
        print('======================================================')
        print(video)
        print('accuracy: {}'.format(np.max((acc_rate, 1-same_rate))))
        print('class num: {0} to {1}'.format(np.sum(pred), len(pred) - np.sum(pred)))
        print('metric? :{}'.format(dist))
        if detected_anomaly:
            print('DETECTED ANOMALY! NOT USING THIS VIDEO')

    return df_tmp

## SORT

In [ ]:
def my_add_sort_label_col(df_pred_video, tracking_video):
    print('Use my add_sort_label_col')
    sort_cluster_label_counts = df_pred_video.groupby('sort_cluster')['label'].value_counts()
    sort_cluster_label_counts = sort_cluster_label_counts.to_frame().rename({'label': 'label_counts'}, axis=1).reset_index()

    sort_clusters = sorted(df_pred_video['sort_cluster'].unique())
    labels_in_frame = sorted(tracking_video['player'].unique())
    df_dist = pd.DataFrame(np.ones((len(sort_clusters), len(labels_in_frame))), columns=labels_in_frame, index=sort_clusters) * 1e-5
    for _, (sort_cluster, label, label_counts) in sort_cluster_label_counts.iterrows():
        df_dist.loc[sort_cluster, label] = label_counts
    df_dist = 1 / df_dist

    df_updated_list = []
    for frame, df_frame in df_pred_video.groupby('frame'):
        dist_matrix = df_dist.loc[df_frame['sort_cluster'].values, :].values
        sort_cluster_idx, label_idx = linear_sum_assignment(dist_matrix)
        df_frame['label_sort'] = df_dist.columns[label_idx]
        df_updated_list.append(df_frame)
    df_pred_video_finalized = pd.concat(df_updated_list)
    return df_pred_video_finalized

In [ ]:
# def add_sort_label_col(out):
#     # Find the top occuring label for each deepsort_cluster
#     sortlabel_map = out.groupby('sort_cluster')['label'].value_counts() \
#         .sort_values(ascending=False).to_frame() \
#         .rename(columns={'label':'label_count'}) \
#         .reset_index() \
#         .groupby(['sort_cluster']) \
#         .first()['label'].to_dict()
#     # Find the # of times that label appears for the deepsort_cluster.
#     sortlabelcount_map = out.groupby('sort_cluster')['label'].value_counts() \
#         .sort_values(ascending=False).to_frame() \
#         .rename(columns={'label':'label_count'}) \
#         .reset_index() \
#         .groupby(['sort_cluster']) \
#         .first()['label_count'].to_dict()

#     out['label_sort'] = out['sort_cluster'].map(sortlabel_map)
#     out['label_count_sort'] = out['sort_cluster'].map(sortlabelcount_map)

#     return out

def apply_sort(df_pred, tracking):
    df_pred_updated_list = []
    game_play2tracking = dict(tuple(tracking.groupby('game_play')))
    for video, df_pred_video in tqdm(df_pred.groupby('video'), total=df_pred['video'].nunique(), desc='Apply SORT'):
        # if np.random.rand() > 0.1:
        #     continue
        game_play = video.split('_')[0] + '_' + video.split('_')[1]
        view = video.split('_')[2].split('.')[0]
        tracking_video = game_play2tracking[game_play] # tracking_processed[tracking_processed['game_play'] == game_play]

        mot_tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
        res = []
        for frame, df_pred_frame in tqdm(df_pred_video.groupby(['frame']), total=df_pred_video['frame'].nunique()):
            df_pred_frame['right'] = df_pred_frame['left'] + df_pred_frame['width']
            df_pred_frame['bottom'] = df_pred_frame['top'] + df_pred_frame['height']
            dets = df_pred_frame[['left', 'top', 'right', 'bottom', 'conf']].values
            trackers = mot_tracker.update(dets)

            df_pred_frame_tracked = pd.DataFrame(trackers, columns=['left', 'top', 'right', 'bottom', 'sort_cluster'])

            df_pred_frame_tracked['x'] = (df_pred_frame_tracked['left'] + df_pred_frame_tracked['right']) / 2
            df_pred_frame_tracked['y'] = (df_pred_frame_tracked['top'] + df_pred_frame_tracked['bottom']) / 2

            dist_matrix = distance.cdist(df_pred_frame[['x', 'y']].values, df_pred_frame_tracked[['x', 'y']].values, metric='euclidean')
            d0_idx, d1_idx = linear_sum_assignment(dist_matrix)

            df_pred_frame_aligned = df_pred_frame_tracked.iloc[d1_idx, :]
            df_pred_frame_aligned.index = df_pred_frame.index[d0_idx]
            df_pred_frame_updated = pd.concat([df_pred_frame.iloc[d0_idx], df_pred_frame_aligned['sort_cluster']], axis=1)
            res.append(df_pred_frame_updated)
        df_pred_video_updated = pd.concat(res)

        # ## OLD VERSION
        # df_pred_video_updated = add_sort_label_col(df_pred_video_updated)
        # df_pred_video_updated['label_sort'] = df_pred_video_updated['label_sort'].fillna(df_pred_video_updated['label'])
        # df_pred_video_updated = df_pred_video_updated.drop('label', axis=1).rename(columns={'label_sort':'label'})

        # df_pred_video_updated = df_pred_video_updated.sort_values('label_count_sort', ascending=False)

        # df_pred_video_updated = df_pred_video_updated.loc[
        #     ~df_pred_video_updated[['video_frame', 'label']].duplicated()]
        # # df_pred_video_updated = drop_duplicates_with_relabeling(df_pred_video_updated)

        ## NEW VERSION
        df_pred_video_updated = my_add_sort_label_col(df_pred_video_updated, tracking_video)
        df_pred_video_updated['label_sort'] = df_pred_video_updated['label_sort'].fillna(df_pred_video_updated['label'])
        df_pred_video_updated = df_pred_video_updated.drop('label', axis=1).rename(columns={'label_sort':'label'})

        df_pred_updated_list.append(df_pred_video_updated)

        if debug:
            eval_video(df_pred_video, labels, video)
            eval_video(df_pred_video_updated, labels, video)
            print('Shape: {0} -> {1}'.format(df_pred_video.shape[0], df_pred_video_updated.shape[0]))
    df_pred_sort = pd.concat(df_pred_updated_list)
    return df_pred_sort

In [ ]:
# def drop_duplicates_with_relabeling(pred_video):
#     pred_video = add_sort_label_col(pred_video)
#     pred_video['label_sort'] = pred_video['label_sort'].fillna(pred_video['label'])
#     pred_video = pred_video.sort_values('label_count_sort', ascending=False)
#     sort_cluster_label_counts = pred_video.groupby('sort_cluster')['label'].value_counts()

#     res_list = []
#     labels_video_set = set(pred_video['label'].unique())
#     for frame, df_pred_frame_tmp in  pred_video.sort_values('frame').groupby('frame'):
#         # Add primaries
#         df_pred_frame_tmp = df_pred_frame_tmp.sort_values('label_count_sort', ascending=False)
#         df_pred_frame_tmp_safe = df_pred_frame_tmp[~df_pred_frame_tmp['label_sort'].duplicated()]
#         df_pred_frame_tmp_remaining = df_pred_frame_tmp[df_pred_frame_tmp['label_sort'].duplicated()]
#         labels_in_use_set = set(df_pred_frame_tmp_safe['label_sort'])
#         labels_left_set = labels_video_set - labels_in_use_set

#         # Add secondaries
#         additionals = []
#         for _, df_remain in df_pred_frame_tmp_remaining.iterrows():
#             for label_sort in dict(sort_cluster_label_counts[df_remain['sort_cluster']]):
#                 if not label_sort in labels_left_set:
#                     continue
#                 df_remain['label_sort'] = label_sort
#                 df_remain['how_sort'] = 'secondary' 
#                 additionals.append(df_remain)
#                 break
#         if len(additionals) > 0:
#             df_additionals = pd.concat(additionals, axis=1).T
#             df_pred_frame_tmp_safe = pd.concat([df_pred_frame_tmp_safe, df_additionals])
#             df_pred_frame_tmp_safe = df_pred_frame_tmp_safe[~df_pred_frame_tmp_safe['label_sort'].duplicated()]
#             labels_left_set -= set(df_additionals['label_sort'])

#         # Add remainings
#         # ここの処理は,clusterが各フレームで被らないことが前提。DeepSORTではおそらくうまくいかないはず。
#         if len(labels_left_set) >= len(df_pred_frame_tmp_remaining):
#             df_pred_frame_tmp_remaining = df_pred_frame_tmp[~df_pred_frame_tmp['sort_cluster'].isin(df_pred_frame_tmp_safe['sort_cluster'])]
#             df_pred_frame_tmp_remaining['label_sort'] = np.random.choice(
#                 list(labels_left_set),
#                 len(df_pred_frame_tmp_remaining),
#                 replace=False
#             )
#             df_pred_frame_tmp_safe = pd.concat([df_pred_frame_tmp_safe, df_pred_frame_tmp_remaining])

#         res_list.append(df_pred_frame_tmp_safe)

#     df_pred_video_updated_added = pd.concat(res_list)
#     df_pred_video_updated_added = df_pred_video_updated_added.drop('label', axis=1).rename(columns={'label_sort':'label'})
#     return df_pred_video_updated_added

## Ghost detection

In [ ]:
def frames_list_to_segment_list(frames):
    segment_list = []
    start_frame = -1
    prev_frame = -1
    for frame in frames:
        if prev_frame == -1:
            prev_frame = frame
        if start_frame == -1:
            start_frame = frame
    
        if frame - prev_frame >= 2:
            segment_list.append([start_frame, prev_frame])
            start_frame = frame

        prev_frame = frame
    segment_list.append([start_frame, prev_frame])
    return segment_list

def get_closest_label(frame, df_pred_video, query_label):
    df_pred_frame = df_pred_video[df_pred_video['frame'] == frame]
    df_pred_frame['dx'] = df_pred_frame['x'] - df_pred_frame[df_pred_frame['label'] == query_label]['x'].values[0]
    df_pred_frame['dy'] = df_pred_frame['y'] - df_pred_frame[df_pred_frame['label'] == query_label]['y'].values[0]
    dist = np.sqrt(df_pred_frame['dx'].values**2 + df_pred_frame['dy']**2)
    dist[dist==0] = 1e9
    closest_id = np.argmin(dist)
    closest_label = df_pred_frame['label'].values[closest_id]
    closest_pos = df_pred_frame[['x', 'y']].values[closest_id]
    relative_pos = df_pred_frame[df_pred_frame['label'] == query_label][['x', 'y']].values[0] - closest_pos
    return closest_label, closest_pos, relative_pos

def get_data(df_video, frame, label):
    return df_video[(df_video['frame'] == frame) & (df_video['label'] == label)]

def get_base_data(df_video, frame0, frame1, target_label):
    df_base = df_video[(df_video['frame'] >= frame0) & (df_video['frame'] <= frame1) & (df_video['label'] == target_label)]
    return df_base

In [ ]:
def add_ghost_simple(df_pred):
    update_list = []

    for video, df_pred_video in tqdm(df_pred.groupby('video')):
        df_pred_video_updated = df_pred_video.copy()

        video_name = video[:-4]

        for label in df_pred_video['label'].unique():
            # if label != 'H90':
            #     continue
            df_pred_video = df_pred_video.sort_values('frame')
            cond_visible = np.array([label in labels.values for _, labels in df_pred_video.groupby('frame')['label']])
            non_visible_frames = df_pred_video['frame'].unique()[~cond_visible]

            non_visible_segment_list = frames_list_to_segment_list(non_visible_frames)
            for seg in non_visible_segment_list:
                closest_labels = [0, 0]
                closest_pos_list = [0, 0]
                relative_pos_list = [0, 0]

                for i in range(len(seg)):
                    frame_tmp = seg[i] - (-1)**i
                    if frame_tmp not in df_pred_video.frame.unique():
                        closest_labels[i] = 'NaN'
                    else:
                        closest_labels[i], closest_pos_list[i], relative_pos_list[i] = get_closest_label(frame_tmp, df_pred_video, label)

                if closest_labels[0] == closest_labels[1] and closest_labels[0] != 'NaN' and np.mean(np.linalg.norm(relative_pos_list, axis=1)) < 30:
                    relative_pos = np.mean(relative_pos_list, axis=0)
                    if np.linalg.norm(relative_pos) == 0:
                        relative_pos = [1, 1]
                    bases_df = get_base_data(df_pred_video, seg[0], seg[1], closest_labels[0])
                    bases_df['label'] = label
                    bases_df['left'] += relative_pos[0]
                    bases_df['top'] += relative_pos[1]
                    update_list.append(bases_df)
    df_pred_updated = pd.concat([df_pred, pd.concat(update_list)])
    df_pred_updated = df_pred_updated.sort_values('frame')
    return df_pred_updated


In [ ]:
def add_ghosts(df_pred, tracking):
    update_list = []

    for video, df_pred_video in tqdm(df_pred.groupby('video')):
        game_play = video.split('_')[0] + '_' + video.split('_')[1]
        tracking_video = tracking[tracking['game_play'] == game_play]

        closest_labels_to_frame = {}
        # Iterate over frames to detect frames that are close enough
        for frame, df_pred_frame in df_pred_video.sort_values('frame').groupby('frame'):
            dist_thres = (df_pred_frame['width'].mean() + df_pred_frame['height'].mean()) / 2 * 1.

            pos_helmets = df_pred_frame[['x', 'y']].values
            dist = distance.cdist(pos_helmets, pos_helmets, metric='euclidean') + np.eye(len(pos_helmets)) * 1e9
            closest_idxs = np.argmin(dist, axis=1)
            for label0_idx, label1_idx in enumerate(closest_idxs):
                if dist[label0_idx, label1_idx] > dist_thres:
                    continue
                if closest_idxs[label1_idx] == label0_idx:
                    label0 = df_pred_frame['label'].values[label0_idx]
                    label1 = df_pred_frame['label'].values[label1_idx]
                    key = sorted([label0, label1])[0] + '_' + sorted([label0, label1])[1]
                    if key not in closest_labels_to_frame.keys():
                        closest_labels_to_frame[key] = set()
                    closest_labels_to_frame[key].add(frame)

        tracking_dist_thres = 1.5
        for label0_label1, frames in closest_labels_to_frame.items():
            label0, label1 = label0_label1.split('_')
            pred_label0 = df_pred_video[df_pred_video['label'] == label0]
            pred_label1 = df_pred_video[df_pred_video['label'] == label1]
            frames = sorted(list(frames))
            non_visible_seg_list = []
            prev_frame_visible = frames[0]
            for frame_visible in frames:
                if frame_visible - prev_frame_visible > 1:
                    non_visible_seg_list.append([prev_frame_visible + 1, frame_visible - 1])
                prev_frame_visible = frame_visible

            for frame0, frame1 in non_visible_seg_list:
                # Check the distance between two tracking data
                tracking_0 = tracking_video[(tracking_video['player'] == label0) & (tracking_video['frame'] >= frame0) & (tracking_video['frame'] <= frame1)][['x', 'y']].values
                tracking_1 = tracking_video[(tracking_video['player'] == label1) & (tracking_video['frame'] >= frame0) & (tracking_video['frame'] <= frame1)][['x', 'y']].values
                if len(tracking_0) != len(tracking_1):
                    continue
                if np.linalg.norm(tracking_0 - tracking_1, axis=1).max() > tracking_dist_thres:
                    continue

               #  print("Complete! frame: {0} to {1}, label: {2} and {3}".format(frame0, frame1, label0, label1))
                relative_pos_0to1 = (pred_label1[pred_label1['frame'] == frame0 - 1][['x', 'y']].values + pred_label1[pred_label1['frame'] == frame1 + 1][['x', 'y']].values) - \
                                                     (pred_label0[pred_label0['frame'] == frame0 - 1][['x', 'y']].values  + pred_label0[pred_label0['frame'] == frame1 + 1][['x', 'y']].values)
                relative_pos_0to1 /= 2
                # if np.linalg.norm(relative_pos_0to1) < 5:
                #    print('here')
                #    relative_pos_0to1 = np.array([[2, 2]])

                # Complement missing label0 with label1
                df_label0_complemented = pred_label1[(pred_label1['frame'] >= frame0) & (pred_label1['frame'] <= frame1)]
                df_label0_complemented['label'] = label0
                df_label0_complemented['left'] -= relative_pos_0to1[0][0]
                df_label0_complemented['top'] -= relative_pos_0to1[0][1]
                df_label0_complemented = df_label0_complemented[~df_label0_complemented['frame'].isin(pred_label0['frame'].values)]

                # Complement missing label0 with label1
                df_label1_complemented = pred_label0[(pred_label0['frame'] >= frame0) & (pred_label0['frame'] <= frame1)]
                df_label1_complemented['label'] = label1
                df_label1_complemented['left'] += relative_pos_0to1[0][0]
                df_label1_complemented['top'] += relative_pos_0to1[0][1]
                df_label1_complemented = df_label1_complemented[~df_label1_complemented['frame'].isin(pred_label1['frame'].values)]

                update_list.append(df_label0_complemented)
                update_list.append(df_label1_complemented)
                # print(relative_pos_0to1)
    df_pred_updated = pd.concat([df_pred, pd.concat(update_list)])
    df_pred_updated = df_pred_updated[~df_pred_updated[['video_frame', 'label']].duplicated()]
    df_pred_updated = df_pred_updated[~df_pred_updated[["video_frame", "left", "width", "top", "height"]].duplicated()]

    return df_pred_updated

## others

In [ ]:
def clip_edges(df):
    df['right'] = df['left'] + df['width']
    df['bottom'] = df['top'] + df['height']
    df[['top', 'bottom']] = np.clip(df[['top', 'bottom']].values.astype(int), 0, 720)
    df[['left', 'right']] = np.clip(df[['left', 'right']].values.astype(int), 0, 1280)
    df['width'] = df['right'] - df['left']
    df['height'] = df['bottom'] - df['top']
    return df

In [ ]:
def generate_video(submission_df, labels):
    submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    debug_videos = submission_df['video'].unique()
    debug_labels = labels.query('video in @debug_videos')
    scorer = NFLAssignmentScorer(debug_labels)
    scorer.score(submission_df)

    # Create video showing predictions for one of the videos.
    video_out = video_with_predictions(
        f'../input/nfl-health-and-safety-helmet-assignment/train/{debug_videos[0]}',
        scorer.sub_labels)
    
    frac = 0.60 # scaling factor for display
    display(Video(data=video_out,
                  embed=True,
                  height=int(720*frac),
                  width=int(1280*frac))
           )
    
def eval_sub(submission_df, labels):
    submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    for video, sub in submission_df.groupby('video'):
        scorer = NFLAssignmentScorer(labels[labels['video'] == video])
        baseline_score = scorer.score(submission_df[submission_df['video'] == video])
        print(f"Score @ {video}: {baseline_score:0.4f}")
    print('============')
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(submission_df)
    print(f"validation score {baseline_score:0.4f}")
    

def save_sub(submission_df, filename, all_cols = False):
    ss = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/sample_submission.csv')

    # Final Checks
    if not all_cols:
        submission_df = submission_df[ss.columns]
    submission_df = submission_df.loc[
        ~submission_df[['video_frame','label']].duplicated()]

    if not all_cols:
        # Condition 4
        submission_df['left'] = submission_df['left'].astype(np.int64)
        submission_df['top'] = submission_df['top'].astype(np.int64)
        submission_df['width'] = submission_df['width'].astype(np.int64)
        submission_df['height'] = submission_df['height'].astype(np.int64)
        submission_df = clip_edges(submission_df)

        # Condition 1
        submission_df = submission_df.loc[~submission_df[['video_frame', 'label']].duplicated()]

        # Condition 2
        submission_df = submission_df.loc[~submission_df[['video_frame', 'left', 'top', 'width', 'height']].duplicated()]

        # Condition 3
        res = []
        for video_frame, df_frame in submission_df.groupby('video_frame'):
            if len(df_frame) > 22:
                df_frame = df_frame.loc[:22]
            res.append(df_frame)
        submission_df = pd.concat(res)
        
        submission_df = submission_df[ss.columns]

        ok_for_submission = check_submission(submission_df)
        if not ok_for_submission:
            raise ValueError
    submission_df.to_csv(filename, index=False)

def eval_video(submission_df, labels, video):
    submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    scorer = NFLAssignmentScorer(labels[labels['video'] == video])
    baseline_score = scorer.score(submission_df[submission_df['video'] == video])
    print(f"validation score @{video}: {baseline_score:0.4f}")
    

# Prepare

In [ ]:
n_test_videos = len(os.listdir('../input/nfl-health-and-safety-helmet-assignment/test/'))
# Run in debug mode unless during submission
if n_test_videos == 6:
    debug = True
else:
    debug = False

# Configurables
n_debug_samples = 1
random_state = 42
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = DIG_STEP*10

# Read in the data.

BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
    video_dir = BASE_DIR + '/train'
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')
    video_dir = BASE_DIR + '/test'

tracking = add_track_features(tracking)

In [ ]:
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    # sample_videos = ['57586_004152_Endzone.mp4', '57586_004152_Sideline.mp4']
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]
    print(sample_videos)
tracking.shape, helmets.shape, labels.shape

In [ ]:
helmets['video'] = helmets['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
helmets['view'] = helmets['video_frame'].str.split('_').str[2].astype(str)
helmets['frame'] = helmets['video_frame'].str.split('_').str[3].astype(np.int64)
helmets['x'] = helmets['left'] + helmets['width'] / 2.0
helmets['y'] = helmets['top'] + helmets['height'] / 2.0

tracking_merged_list = []
for video, df_pred_video in tqdm(helmets.groupby('video')):
    game_play = video.split('_')[0] + '_' + video.split('_')[1]
    tracking_video = tracking[tracking['game_play'] == game_play]
    est_frame_to_tracking_video = dict(list(tracking_video.groupby('est_frame')))
    keys = np.array(list(est_frame_to_tracking_video.keys()))
    for frame_id in df_pred_video['frame'].unique():
        opt_idx = np.argmin(np.abs(keys - frame_id))
        tmp = est_frame_to_tracking_video[keys[opt_idx]].copy()
        tmp['frame'] = frame_id
        tracking_merged_list.append(tmp)
tracking_processed = pd.concat(tracking_merged_list)

In [ ]:
# ICP
LINE_PER_HELMET_CAND_SIDELINE = np.arange(8, 10, 0.1) # これがendzoneと違うのは、おそらく下記のscale factorが違うせい？
LINE_PER_HELMET_CAND_ENDZONE = np.arange(4, 5, 0.1)

# Monte-carlo ICP
NUM_PARTICLES = 100
RATE_X2Y = {'Endzone': 0.5, 'Sideline': 1.0}
SCALE_FACTOR = {'Endzone': [2, 4.5], 'Sideline': [4, 10]}
TEAM_DIF_PENALTY = 2.0
NUM_ITER_OUTSIDERS = 6
SCALE_RATE_EXP = 0.7
NUM_HELMETS_THRES_FOR_OUTSIDERS = 24

# Team clusterization
CLUSTERIZATION_RATE_THRES = 1e9
CLUSTERIZATION_DIST_THRES = 0
CLUSTERIZATION_HELMET_SHRINK_RATE = 0.7

# SORT
MAX_AGE = 1
MIN_HITS = 3
IOU_THRESHOLD = 0.3

# ICP

In [ ]:
df_results_icp = apply_icp_multiprocess_by_team(helmets, tracking_processed, video_dir)

In [ ]:
if debug:
    eval_sub(df_results_icp, labels)
save_sub(df_results_icp, 'submission_icp.csv', all_cols=True)
# save_sub(df_results_icp, 'submission.csv')

# SORT

In [ ]:
df_results_sort = apply_sort(df_results_icp, tracking_processed)

In [ ]:
if debug:
    eval_sub(df_results_sort, labels)
save_sub(df_results_sort, 'submission_sort.csv', all_cols=True)
# save_sub(df_results_deepsort, 'submission.csv')

# Ghost Detection

In [ ]:
df_results_simple_ghosts = add_ghost_simple(df_results_sort)
save_sub(df_results_simple_ghosts, 'submission_few_ghosts.csv', all_cols=True)

df_results_plenty_ghosts = add_ghosts(df_results_simple_ghosts, tracking_processed)
save_sub(df_results_plenty_ghosts, 'submission_plenty_ghosts.csv', all_cols=True)

In [ ]:
if debug:
    eval_sub(df_results_plenty_ghosts, labels)
save_sub(df_results_plenty_ghosts, 'submission.csv')

# Video

In [ ]:
# if debug:
#     generate_video(df_results_plenty_ghosts, labels)